In [1]:
import numpy as np
import os
import librosa as lb
import soundfile as sf
import museval
from matplotlib import pyplot as plt
from tqdm import tqdm
import librosa.display

In [86]:
def random_gen():
    k = np.round(np.random.uniform(0.05, 0.31), 3)
    return k
def random_diag():
    k = np.round(np.random.uniform(0.6, 1), 3)
    return k

def gen_A():
    A = np.array([[random_diag(), random_gen(), random_gen(), random_gen()],
                  [random_gen(), random_diag(), random_gen(), random_gen()],
                  [random_gen(), random_gen(), random_diag(), random_gen()],
                  [random_gen(), random_gen(), random_gen(), random_diag()]])
    return np.round(A, 2)

In [111]:
def conditions(vx, bx, dx, block):
    flag = True
    if vx.shape[0] != block:
        flag = False
    if np.linalg.norm(vx) <= 0.9 or np.linalg.norm(bx) <= 0.9 or np.linalg.norm(dx) <= 0.9:
        flag = False
    return flag

In [113]:
clean_path = "/home/rajesh/Desktop/Datasets/musdb18hq/train/"
outpath = "/home/rajesh/Desktop/Datasets/musdb18hq_extensivebleed/"

In [114]:
files = sorted(os.listdir(clean_path))

for file in tqdm(files):
    vocal, fs = lb.load(clean_path+file+"/vocals.wav")
    bass, fs = lb.load(clean_path+file+"/bass.wav")
    drums, fs = lb.load(clean_path+file+"/drums.wav")
    other, fs = lb.load(clean_path+file+"/other.wav")
    
    path = outpath+file
    os.makedirs(path, exist_ok=True)
    os.makedirs(path+'/Audio', exist_ok=True)
    os.makedirs(path+'/Numpy', exist_ok=True)
    A = gen_A()
    np.save(path+'/mixing.npy', A)
    
    sec = 10
    block = sec*fs
    length = vocal.shape[0]
    for i in range(0, length, block):
        vx = vocal[i:i+block]
        bx = bass[i:i+block]
        dx = drums[i:i+block]
        ox = other[i:i+block]

        if conditions(vx, bx, dx, block):

            fapath = path+'/Audio/{}_{}/'.format(int(i/fs), int(i/fs)+sec)
            fnpath = path+'/Numpy/{}_{}/'.format(int(i/fs), int(i/fs)+sec)
            os.makedirs(fapath, exist_ok=True)
            os.makedirs(fnpath, exist_ok=True)

            np.save(fnpath+'vocal.npy', vx)
            np.save(fnpath+'bass.npy', bx)
            np.save(fnpath+'drums.npy', dx)
            np.save(fnpath+'other.npy', ox)

            s = np.array([vx, bx, dx, ox])
            x = A @ s

            np.save(fnpath+'bvocal.npy', x[0])
            np.save(fnpath+'bbass.npy', x[1])
            np.save(fnpath+'bdrums.npy', x[2])
            np.save(fnpath+'bdrums.npy', x[3])

            sf.write(fapath+'vocal.wav', vx, fs)
            sf.write(fapath+'bass.wav', bx, fs)
            sf.write(fapath+'drums.wav', dx, fs)
            sf.write(fapath+'other.wav', ox, fs)

            sf.write(fapath+'bvocal.wav', x[0], fs)
            sf.write(fapath+'bbass.wav', x[1], fs)
            sf.write(fapath+'bdrums.wav', x[2], fs)
            sf.write(fapath+'bother.wav', x[3], fs)

100%|██████████████████████████████████████████████████████████████| 100/100 [33:33<00:00, 20.13s/it]
